<h1>BSP412 -- Using PuLP with pandas</h1>

<p>pandas is a Python library for data manipulation. It is useful whenever you need to read data from external files and make it readable for Python functions. In this tutorial we first use pandas to read external data into a useable format and then show how PuLP can be used to solve a special kind of an LP problem, a scheduling problem.</p>
<p>The useable format in this case is an object called a data frame. You will run into data frames if you study other languages that are popular among data scientists, such as R. A data frame presents data grouped in rows and columns. </p>

In [1]:
# http://benalexkeen.com/linear-programming-with-python-and-pulp-part-5/
import pandas as pd
import pulp

# pandas is not included in jupyter http://nikgrozev.com/2015/12/27/pandas-in-jupyter-quickstart-and-useful-snippets/

<p>Let us read the csv (comma separated file type) data into a data frame that is convenient to handle in Python functions.</p>
<p>We group the data that is read from the csv file to months and factories.</p>

In [2]:
factories = pd.DataFrame.from_csv('factory_variables.csv', index_col=['Month', 'Factory'])
demand = pd.DataFrame.from_csv('monthly_demand.csv', index_col=['Month'])

<p>The data shows that we have 2 factories and we know what are the monthly capacities and costs of each factory:</p>

In [3]:
factories

Max_Capacity  Min_Capacity  Variable_Costs  Fixed_Costs
Month Factory                                                         
1     A              100000         20000              10          500
      B               50000         20000               5          600
2     A              110000         20000              11          500
      B               55000         20000               4          600
3     A              120000         20000              12          500
      B               60000         20000               3          600
4     A              145000         20000               9          500
      B              100000         20000               5          600
5     A              160000         20000               8          500
      B                   0             0               0            0
6     A              140000         20000               8          500
      B               70000         20000               6          600
7     A              155000         20000               5          500
      B               60000         20000               4          600
8     A              200000         20000               7          500
      B              100000         20000               6          600
9     A              210000         20000               9          500
      B              100000         20000               8          600
10    A              197000         20000              10          500
      B              100000         20000              11          600
11    A               80000         20000               8          500
      B              120000         20000              10          600
12    A              150000         20000               8          500
      B              150000         20000              12          600

<p>As you can see, factory B is in maintenance on month 5.</p>

<p>We also know the demands for our products for each month (this is also a data frame albeit a very simple one):</p>

In [4]:
demand

,Demand
Month,
1,120000
2,100000
3,130000
4,130000
5,140000
6,130000
7,150000
8,170000
9,200000


<p>Because we have information on costs, we clearly have a minimisation problem at hand. Therefore a sensible decision variable would be to indicate whether a factory is in production or not. However, as we can vary the production in each factory within the given limits, we can also have a decision variable on how much to produce in each factory.</p>
<p>In the below code, we write production as an integer variable with only a lower bound of 0, and factory status as a binary variable (i.e. with value 0 or 1).</p>

In [15]:
production = pulp.LpVariable.dicts("production",
                                     ((month, factory) for month, factory in factories.index),
                                     lowBound=0,
                                     cat='Integer')
factorystatus = pulp.LpVariable.dicts("factorystatus",
                                     ((month, factory) for month, factory in factories.index),
                                     cat='Binary')


<p>Then we tell PuLP what to do in the LP problem.</p>

In [16]:
schedulingproblem = pulp.LpProblem("Scheduling problem with cost minimisation", pulp.LpMinimize)

<p>The objective function consists of costs that are to be minimised. The total fixed costs are equal to the factory status (0 not working, 1 working) times the fixed costs, summed over all months. The total variable costs are equal to the production times the variable costs, summed over all months.</p>
<p>We now add the objective function into the model.</p>

In [17]:
schedulingproblem += pulp.lpSum(
    [production[month, factory] * factories.loc[(month, factory), 'Variable_Costs'] for month, factory in factories.index]
    + [factorystatus[month, factory] * factories.loc[(month, factory), 'Fixed_Costs'] for month, factory in factories.index]
)

<p>We then add constraints into the model. These say that each month production must include demand.</p>

In [18]:
months = demand.index
for month in months:
    schedulingproblem += production[(month, 'A')] + production[(month, 'B')] == demand.loc[month, 'Demand']

<p>We also need to tell PuLP that production must not exceed the maximum capacity or be less than the minimum capacity, and that if the factory status is 0, then production must be 0 as well.</p>

In [19]:
# Production in any month must be between minimum and maximum capacity, or zero.
for month, factory in factories.index:
    min_production = factories.loc[(month, factory), 'Min_Capacity']
    max_production = factories.loc[(month, factory), 'Max_Capacity']
    schedulingproblem += production[(month, factory)] >= min_production * factorystatus[month, factory]
    schedulingproblem += production[(month, factory)] <= max_production * factorystatus[month, factory]

<p>Final constraint is that factory B is off in month 5. This constraint is directly for both decision variables (status and production).</p>

In [20]:
model += factorystatus[5, 'B'] == 0
model += production[5, 'B'] == 0

<p>This is what the problem looks like now:</p>

In [21]:
schedulingproblem

Scheduling problem with cost minimisation:
MINIMIZE
500*factorystatus_(1,_'A') + 600*factorystatus_(1,_'B') + 500*factorystatus_(10,_'A') + 600*factorystatus_(10,_'B') + 500*factorystatus_(11,_'A') + 600*factorystatus_(11,_'B') + 500*factorystatus_(12,_'A') + 600*factorystatus_(12,_'B') + 500*factorystatus_(2,_'A') + 600*factorystatus_(2,_'B') + 500*factorystatus_(3,_'A') + 600*factorystatus_(3,_'B') + 500*factorystatus_(4,_'A') + 600*factorystatus_(4,_'B') + 500*factorystatus_(5,_'A') + 500*factorystatus_(6,_'A') + 600*factorystatus_(6,_'B') + 500*factorystatus_(7,_'A') + 600*factorystatus_(7,_'B') + 500*factorystatus_(8,_'A') + 600*factorystatus_(8,_'B') + 500*factorystatus_(9,_'A') + 600*factorystatus_(9,_'B') + 10*production_(1,_'A') + 5*production_(1,_'B') + 10*production_(10,_'A') + 11*production_(10,_'B') + 8*production_(11,_'A') + 10*production_(11,_'B') + 8*production_(12,_'A') + 12*production_(12,_'B') + 11*production_(2,_'A') + 4*production_(2,_'B') + 12*production_(3,_'A') 

<p>And now we can solve it.</p>

In [23]:
schedulingproblem.solve()
pulp.LpStatus[schedulingproblem.status]

'Optimal'

<p>We see that, for example, factory A produces 70000 units in month 1, which is 70% of its max capacity.</p>

In [26]:
production[1,'A'].varValue

70000.0

<p>And here is a way to automate the display of the production schedule via pandas data frames. Basically, we just loop over months and factories and write the varValue outputs into a neat data frame.</p>

In [27]:
output = []
for month, factory in production:
    var_output = {
        'Month': month,
        'Factory': factory,
        'Production': production[(month, factory)].varValue,
        'Factory Status': factorystatus[(month, factory)].varValue
    }
    output.append(var_output)
output_df = pd.DataFrame.from_records(output).sort_values(['Month', 'Factory'])
output_df.set_index(['Month', 'Factory'], inplace=True)
output_df

Factory Status  Production
Month Factory                            
1     A                   1.0     70000.0
      B                   1.0     50000.0
2     A                   1.0     45000.0
      B                   1.0     55000.0
3     A                   1.0     70000.0
      B                   1.0     60000.0
4     A                   1.0     30000.0
      B                   1.0    100000.0
5     A                   1.0    140000.0
      B                   NaN         0.0
6     A                   1.0     60000.0
      B                   1.0     70000.0
7     A                   1.0     90000.0
      B                   1.0     60000.0
8     A                   1.0     70000.0
      B                   1.0    100000.0
9     A                   1.0    100000.0
      B                   1.0    100000.0
10    A                   1.0    190000.0
      B                   0.0         0.0
11    A                   1.0     80000.0
      B                   1.0     60000.0
12    A                   1.0    100000.0
      B                   0.0         0.0

<p>The total costs are:</p>

In [30]:
pulp.value(schedulingproblem.objective)

12906400.0